In [125]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Задание 1

Какова вероятность отправиться на прогулку если идёт дождь, при наличии следующих наблюдений?

```
data = [
        ('солнечно', True),
        ('снег', False),
        ('облачно', False),
        ('дождь', False),
        ('солнечно', True),
        ('снег', False),
        ('облачно', True),
        ('снег', False),
        ('солнечно', False),
        ('облачно', True),
        ('снег', True),
        ('солнечно', True),
        ('дождь', False),
        ('дождь', True),
        ('облачно', True),
]
```

In [126]:
data = [
        ('солнечно', True),
        ('снег', False),
        ('облачно', False),
        ('дождь', False),
        ('солнечно', True),
        ('снег', False),
        ('облачно', True),
        ('снег', False),
        ('солнечно', False),
        ('облачно', True),
        ('снег', True),
        ('солнечно', True),
        ('дождь', False),
        ('дождь', True),
        ('облачно', True),
]
df = pd.DataFrame(data, columns=['weather', 'stroll'])
df

,weather,stroll
0,солнечно,True
1,снег,False
2,облачно,False
3,дождь,False
4,солнечно,True
5,снег,False
6,облачно,True
7,снег,False
8,солнечно,False
9,облачно,True


In [127]:
p_rain =  len(df.loc[df['weather'] == 'дождь']) / len(df)
p_stroll = len(df.loc[df['stroll'] == True]) / len(df)
p_rain_if_stroll = len(df.loc[(df['weather'] == 'дождь') & (df['stroll'] == True)]) / len(df.loc[df['stroll'] == True])
p_stroll_if_rain = (p_stroll*p_rain_if_stroll)/p_rain
print(f"Вероятность отправиться на прогулку если идёт дождь: {p_stroll_if_rain}")

Вероятность отправиться на прогулку если идёт дождь: 0.3333333333333333


## Задание 2

- загрузите датасет для классификации, проведите предобработку, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- Масштабируйте числовые признаки
- В случае дисбаланса проведите балансировку
- решите задачу классификации на ваших данных с использованием рассмотренных моделей sklearn: knn, NB, Logistic Regression, SVM. Не забудьте подобрать гиперпараметры.
- вычислите значения метрик Accuracy, Precision, Recall, F1, ROC AUC score


In [128]:
data= pd.read_csv("../data/csgo_task.csv")
data

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,5.0,5.0
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,650.0,0.0,0.0,1.0,5.0,5.0
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,1.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,de_train,True,200.0,242.0,195.0,359.0,100.0,5950.0,2.0,4.0,1.0,2.0,4.0
122406,174.93,11.0,15.0,de_train,False,500.0,500.0,95.0,175.0,11500.0,23900.0,1.0,2.0,1.0,5.0,5.0
122407,114.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,6700.0,3.0,5.0,1.0,5.0,5.0
122408,94.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,6700.0,3.0,5.0,1.0,5.0,5.0


In [129]:
data.isnull().any().sum()

7

In [130]:
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.dtypes

time_left           float64
ct_score            float64
t_score             float64
map                  object
bomb_planted           bool
ct_health           float64
t_health            float64
ct_armor            float64
t_armor             float64
ct_money            float64
t_money             float64
ct_helmets          float64
t_helmets           float64
ct_defuse_kits      float64
ct_players_alive    float64
t_players_alive     float64
dtype: object

In [131]:
data['bomb_planted'] = (~data['bomb_planted']).astype(np.byte)
d = data.groupby(['map', 'bomb_planted'])['bomb_planted'].count()
d

map          bomb_planted
de_cache     0                  10
             1                 128
de_dust2     0                2593
             1               17766
de_inferno   0                2984
             1               19062
de_mirage    0                2003
             1               15016
de_nuke      0                1780
             1               15436
de_overpass  0                1305
             1               11638
de_train     0                1360
             1               11003
de_vertigo   0                1172
             1                9215
Name: bomb_planted, dtype: int64

In [132]:
data_new = pd.get_dummies(data, columns=['map'])
data_new

,time_left,ct_score,t_score,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,...,ct_players_alive,t_players_alive,map_de_cache,map_de_dust2,map_de_inferno,map_de_mirage,map_de_nuke,map_de_overpass,map_de_train,map_de_vertigo
0,175.00,0.0,0.0,1,500.0,500.0,0.0,0.0,4000.0,4000.0,...,5.0,5.0,0,1,0,0,0,0,0,0
1,156.03,0.0,0.0,1,500.0,500.0,400.0,300.0,600.0,650.0,...,5.0,5.0,0,1,0,0,0,0,0,0
2,96.03,0.0,0.0,1,391.0,400.0,294.0,200.0,750.0,500.0,...,4.0,4.0,0,1,0,0,0,0,0,0
3,76.03,0.0,0.0,1,391.0,400.0,294.0,200.0,750.0,500.0,...,4.0,4.0,0,1,0,0,0,0,0,0
4,174.97,1.0,0.0,1,500.0,500.0,192.0,0.0,18350.0,10750.0,...,5.0,5.0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,0,200.0,242.0,195.0,359.0,100.0,5950.0,...,2.0,4.0,0,0,0,0,0,0,1,0
122406,174.93,11.0,15.0,1,500.0,500.0,95.0,175.0,11500.0,23900.0,...,5.0,5.0,0,0,0,0,0,0,1,0
122407,114.93,11.0,15.0,1,500.0,500.0,495.0,475.0,1200.0,6700.0,...,5.0,5.0,0,0,0,0,0,0,1,0
122408,94.93,11.0,15.0,1,500.0,500.0,495.0,475.0,1200.0,6700.0,...,5.0,5.0,0,0,0,0,0,0,1,0


In [133]:
data = data_new.drop(['ct_helmets', 't_helmets', 'ct_defuse_kits', 'bomb_planted', 'ct_armor', 't_armor'], axis=1)
X = data.values
X[:5]

array([[1.7500e+02, 0.0000e+00, 0.0000e+00, 5.0000e+02, 5.0000e+02,
        4.0000e+03, 4.0000e+03, 5.0000e+00, 5.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [1.5603e+02, 0.0000e+00, 0.0000e+00, 5.0000e+02, 5.0000e+02,
        6.0000e+02, 6.5000e+02, 5.0000e+00, 5.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [9.6030e+01, 0.0000e+00, 0.0000e+00, 3.9100e+02, 4.0000e+02,
        7.5000e+02, 5.0000e+02, 4.0000e+00, 4.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [7.6030e+01, 0.0000e+00, 0.0000e+00, 3.9100e+02, 4.0000e+02,
        7.5000e+02, 5.0000e+02, 4.0000e+00, 4.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [1.7497e+02, 1.0000e+00, 0.0000e+00, 5.0000e+

In [134]:
y = data_new['bomb_planted'].values
y[0:5]

array([1, 1, 1, 1, 1], dtype=int8)

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [136]:
standardScaler = StandardScaler()
standardScaler.fit(X_train)
X_train_std = pd.DataFrame(standardScaler.transform(X_train), columns=data.columns)
X_test_std = pd.DataFrame(standardScaler.transform(X_test), columns=data.columns)
X_train_std.describe()

,time_left,ct_score,t_score,ct_health,t_health,ct_money,t_money,ct_players_alive,t_players_alive,map_de_cache,map_de_dust2,map_de_inferno,map_de_mirage,map_de_nuke,map_de_overpass,map_de_train,map_de_vertigo
count,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04
mean,-6.875034e-13,8.268185e-17,-6.459767e-17,-1.610993e-17,-6.420281e-17,-6.199165e-17,4.398643e-17,3.209351e-16,2.032694e-16,1.563611e-17,-9.476430e-18,3.241729e-17,3.395721e-17,-3.020612e-17,-2.842929e-18,9.555400e-18,-9.938406e-17
std,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00
min,-1.773981e+00,-1.413828e+00,-1.416531e+00,-3.043129e+00,-2.804303e+00,-8.875780e-01,-9.425297e-01,-3.461050e+00,-3.383104e+00,-3.561758e-02,-4.701842e-01,-4.935452e-01,-4.225456e-01,-4.247536e-01,-3.609639e-01,-3.518337e-01,-3.183996e-01
25%,-7.384125e-01,-7.887578e-01,-7.954286e-01,-5.730869e-01,-6.534106e-01,-7.723334e-01,-8.075578e-01,-1.939539e-01,-1.821895e-01,-3.561758e-02,-4.701842e-01,-4.935452e-01,-4.225456e-01,-4.247536e-01,-3.609639e-01,-3.518337e-01,-3.183996e-01
50%,1.548306e-02,-1.636871e-01,-1.743257e-01,6.880525e-01,7.146978e-01,-3.556799e-01,-3.085709e-01,6.228201e-01,6.180392e-01,-3.561758e-02,-4.701842e-01,-4.935452e-01,-4.225456e-01,-4.247536e-01,-3.609639e-01,-3.518337e-01,-3.183996e-01
75%,3.946948e-01,6.697404e-01,8.608458e-01,6.880525e-01,7.217499e-01,4.465999e-01,5.626112e-01,6.228201e-01,6.180392e-01,-3.561758e-02,-4.701842e-01,-4.935452e-01,-4.225456e-01,-4.247536e-01,-3.609639e-01,-3.518337e-01,-3.183996e-01
max,1.527425e+00,5.253592e+00,5.415600e+00,6.880525e-01,1.426960e+00,6.204398e+00,5.601561e+00,6.228201e-01,1.418268e+00,2.807602e+01,2.126826e+00,2.026157e+00,2.366609e+00,2.354306e+00,2.770360e+00,2.842252e+00,3.140707e+00


In [137]:
minMaxScaler = MinMaxScaler()
minMaxScaler.fit(X_train)
X_train_mms = pd.DataFrame(standardScaler.transform(X_train), columns=data.columns)
X_test_mms = pd.DataFrame(standardScaler.transform(X_test), columns=data.columns)
X_train_mms.describe()

,time_left,ct_score,t_score,ct_health,t_health,ct_money,t_money,ct_players_alive,t_players_alive,map_de_cache,map_de_dust2,map_de_inferno,map_de_mirage,map_de_nuke,map_de_overpass,map_de_train,map_de_vertigo
count,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04,8.997600e+04
mean,-6.875034e-13,8.268185e-17,-6.459767e-17,-1.610993e-17,-6.420281e-17,-6.199165e-17,4.398643e-17,3.209351e-16,2.032694e-16,1.563611e-17,-9.476430e-18,3.241729e-17,3.395721e-17,-3.020612e-17,-2.842929e-18,9.555400e-18,-9.938406e-17
std,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00
min,-1.773981e+00,-1.413828e+00,-1.416531e+00,-3.043129e+00,-2.804303e+00,-8.875780e-01,-9.425297e-01,-3.461050e+00,-3.383104e+00,-3.561758e-02,-4.701842e-01,-4.935452e-01,-4.225456e-01,-4.247536e-01,-3.609639e-01,-3.518337e-01,-3.183996e-01
25%,-7.384125e-01,-7.887578e-01,-7.954286e-01,-5.730869e-01,-6.534106e-01,-7.723334e-01,-8.075578e-01,-1.939539e-01,-1.821895e-01,-3.561758e-02,-4.701842e-01,-4.935452e-01,-4.225456e-01,-4.247536e-01,-3.609639e-01,-3.518337e-01,-3.183996e-01
50%,1.548306e-02,-1.636871e-01,-1.743257e-01,6.880525e-01,7.146978e-01,-3.556799e-01,-3.085709e-01,6.228201e-01,6.180392e-01,-3.561758e-02,-4.701842e-01,-4.935452e-01,-4.225456e-01,-4.247536e-01,-3.609639e-01,-3.518337e-01,-3.183996e-01
75%,3.946948e-01,6.697404e-01,8.608458e-01,6.880525e-01,7.217499e-01,4.465999e-01,5.626112e-01,6.228201e-01,6.180392e-01,-3.561758e-02,-4.701842e-01,-4.935452e-01,-4.225456e-01,-4.247536e-01,-3.609639e-01,-3.518337e-01,-3.183996e-01
max,1.527425e+00,5.253592e+00,5.415600e+00,6.880525e-01,1.426960e+00,6.204398e+00,5.601561e+00,6.228201e-01,1.418268e+00,2.807602e+01,2.126826e+00,2.026157e+00,2.366609e+00,2.354306e+00,2.770360e+00,2.842252e+00,3.140707e+00
